In [3]:
import matplotlib
%matplotlib tk
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%autosave 180

import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import sys
sys.path.append('..')

# visualize results module
from visualize import visualize_svm as Visualize


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


<ipython-input-3-0673dc4e64b0>:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [36]:
#####################################################
#####################################################
#####################################################
def get_session_trials_and_paw_movements(animal_id,
                                         n_sec_lockout):
    
    sample_rate_video =15

    # 
    v = Visualize.Visualize()
    v.main_dir = '/media/cat/4TBSSD/yuki/'
    v.animal_id = animal_id

    # 
    v.session_id = "all"  # use a random date as dummy to load all other sessions
    v.get_fname()

    #
    rew = []
    right_paw = []
    t = []
    
    # 
    for ctr_s, session in enumerate(v.sessions):

       
        # 
        try:
 # 
            fname = os.path.join(v.main_dir,
                             v.animal_id,
                             'tif_files',
                             session,
                             session+'_0secNoMove_movements.npz'
                            )

            data = np.load(fname, allow_pickle=True)
            feature_initiations = data['feature_initiations']
            feature_quiescent = data['feature_quiescent']
            labels = data['labels']
            video_shift = data['video_shift']
            label_id = 1
            right_paw_initiations = feature_initiations[label_id]
        except:
            print ("no movements file", fname)
            continue 
    

        # load rewarded lever pulls
        try:
            rewarded = np.loadtxt(os.path.join(v.main_dir,
                                               v.animal_id,
                                               'tif_files',
                                                session,
                                               'rewarded_times.txt'))
        except:
            continue

        #######
        if rewarded.ndim==0 or rewarded.shape[0]<2 or rewarded.shape[0]>400:
            continue
            
        #    
        print ("right paw inits: ", right_paw_initiations)
        
        # find all right-paw mvements
        idx = np.where(right_paw_initiations>0.5)[0]

        #
        n_right_paw = 0
        last_init_ctr = 0
        ctr=1
        while ctr < idx.shape[0]:
            
            # check if in new bout
            if idx[ctr]-idx[ctr-1]>1:
                
                # new bout
                while ctr < idx.shape[0]:
                    if (idx[ctr] - idx[last_init_ctr])>= (n_sec_lockout*sample_rate_video):

                        last_init_ctr = ctr
                        n_right_paw+=1
                        break
                    else:
                        ctr+=1
            ctr+=1
            
        #
        t.append(ctr_s)
        rew.append(rewarded.shape[0])
        right_paw.append(n_right_paw)
        
    return np.hstack(t), np.hstack(rew), np.hstack(right_paw)

# 
def fit_line(x,y):
    from sklearn import datasets, linear_model
    import scipy
    
    
    #print ("x: ", x)
    #print ("y: ", y)
    corr = scipy.stats.pearsonr(x,y.squeeze())
    #print ("pearson corr: ", corr)
    
    plt.title("pcor: "+str(round(corr[0],5))+ 
              "  pval: "+str(round(corr[1],5)))

    # Create linear regression object
    regr = linear_model.LinearRegression()

    #
    x = x[:,None]
    y = y#[:,None]
    regr.fit(x, y)

    # Make predictions using the testing set
    x_test = x 
    y_test = regr.predict(x_test)

    #

    return x_test, y_test

# 
def load_right_paw_traces(n_sec,
                          root_dir, 
                          animal_id, 
                          session_id):
    
    # 
    #session_id = sessions[0]
    print ("Loading session: ", session_id)
    
    try:
        fname = os.path.join(root_dir, 
                             animal_id,
                             'tif_files',
                             session_id,
                             session_id + '_0secNoMove_movements.npz')

        data = np.load(fname, allow_pickle=True)
        feature_initiations = data['feature_initiations']
        feature_quiescent = data['feature_quiescent']
        labels = data['labels']
        video_shift = data['video_shift']
        #print ("video shifts: ", video_shift)
        #print ("labels: ", labels)
        label_id = 1
        #print (feature_initiations.shape)
        right_paw_initiations = feature_initiations[label_id]
    except:
        print ("no movements file")
        return None


In [47]:
########################################################
########################################################   
########################################################

#animal_id = 'AQ2'

animal_ids = ["IA1", 'IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ["AQ2"]

#
n_sec_lockout = 1

# 
for animal_id in animal_ids:

    t, rew, right_paw = get_session_trials_and_paw_movements(animal_id,
                                                            n_sec_lockout)


    print (t.shape)
    print (rew.shape)
    print (right_paw.shape)

    ratio = rew/right_paw

    t = np.arange(len(t))
    
#     if False:
#         t=t[4:]
#         rew=rew[4:]
#         nonrew=nonrew[4:]

#     if False:
#         idx = np.where(ratio<5)[0]
#         rew = rew[idx]
#         nonrew = nonrew[idx]
#         t = t[idx]
#         ratio = ratio[idx]

    #
    ymax = 410
    #
    plt.figure(figsize=(15,5))
    ax=plt.subplot(1,3,1)

    # 
    plt.scatter(t, rew,
                s=200,
               c='blue',
               alpha=.8)
    x, y = fit_line(t, rew)
    #plt.ylim(0,ymax)
    plt.plot(x, y,
             c='blue',
             linewidth=3)
    plt.xlim(t[0], t[-1])

    ###############################
    ax=plt.subplot(1,3,2)
    plt.scatter(t, right_paw,
                s=200,
               c='red',
               alpha=0.8)
    x, y = fit_line(t, right_paw)
    plt.ylim(0,ymax)
    plt.xlim(t[0], t[-1])
    plt.plot(x, y,
             c='red',
             linewidth=3)

    #####################################
    ax=plt.subplot(1,3,3)
    plt.scatter(t, ratio,
                s=200,
                c='green')
    x, y = fit_line(t, ratio)
    plt.plot(x, y,
             c='green',
             linewidth=3,
             alpha=.8)
    plt.xlim(t[0], t[-1])

    plt.suptitle(animal_id)

    if True:
        plt.savefig('/home/cat/ratios_right_paw_to_lever_'+animal_id+'.svg')
        plt.close()
    else:
        plt.show()

 COULDN't FIND SESSION...
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 1. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 1. 0. 0.]
no movements file /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb18_30Hz/IA1pm_Feb18_30Hz_0secNoMove_movements.npz
no movements file /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb19_30Hz/IA1pm_Feb19_30Hz_0secNoMove_movements.npz
no movements file /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz/IA1pm_Feb22_30Hz_0secNoMove_movements.npz
no movements file /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb23_30Hz/IA1pm_Feb23_30Hz_0secNoMove_movements.npz
no movements file /media/cat/4TBSSD/yuki/IA1/tif_files/I

<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Apr22_Week2_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,


right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
no movements file /media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_May4_Week4_30Hz/IA1am_May4_Week4_30Hz_0secNoMove_movements.npz
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]


<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_May2_Week4_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,


right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
(27,)
(27,)
(27,)
 COULDN't FIND SESSION...
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]


...

right paw inits:  [0. 0. 0. ... 0. 0. 0.]


<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Jan21_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,
<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan21_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,


right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]


<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan25_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,
<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Jan26_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,
<ipython-input-36-7fab55b5c811>:51: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan26_30Hz/rewarded_times.txt"
  rewarded = np.loadtxt(os.path.join(v.main_dir,


right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 1. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 1. 1. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 1. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 0. ... 0. 0. 0.]
right paw inits:  [0. 0. 1. ... 1. 0. 0.]
no movements file /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb18_30Hz/AQ2am_Feb18_30Hz_0secNoMove_movements.npz
no movements file /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Feb19_30Hz/AQ2am_Feb19_30Hz_0secNoMove_movements.npz
no movements file /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2